In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [ ]:
load_dotenv()

llm = ChatOpenAI()

In [ ]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [ ]:
def generate_joke(state: JokeState):
    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return{'joke': response}

In [ ]:
def generate_explanation(state: JokeState):
    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt)

    return{'explanation': response}

In [ ]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()


workflow = graph.compile(checkpointer= checkpointer)

workflow

In [ ]:
config1 = {'configurable': {'thread_id': '1'}}

initialstate = {
    'topic': f"Indian Railways"
}

result = workflow.invoke(initialstate, config=config1)

In [ ]:
workflow.get_state(config1)